In [176]:
import pandas as pd
import numpy as np
from darts import TimeSeries
#from darts.models import AutoARIMA  
from sktime.forecasting.arima import AutoARIMA

# Define the directory path and file name
sim_dir = r'C:/Users/Work/OneDrive/GAU/3. Semester/Statistisches Praktikum/Git/NEW_Ensemble_Techniques_TS_FC/project/data/simulations/'
file_name = 'noisy_simdata.csv'

# Combine the directory path and file name
file_path = sim_dir + file_name

# Read and preprocess Dataset
df = pd.read_csv(file_path)
df['Date'] = pd.PeriodIndex(pd.to_datetime(df['Date'], format='%d-%m-%Y'), freq='M')
df.set_index('Date', inplace=True)

# Now you can work with the data DataFrame
# For example, you can print the first few rows
print(df.head())

                 y          x1         x2          x3
Date                                                 
2004-01  50.840469  427.595799  55.337904  900.325291
2004-02  52.871538  434.062163  54.959155  900.775888
2004-03  53.769316  453.264284  56.470633  899.510058
2004-04  57.672973  459.367523  56.704233  903.524834
2004-05  57.182051  462.354356  61.557907  905.071762


In [68]:
df.shape[1]

4

In [177]:
# The following data preprocessing should be above the final wrapper function


# Convert DataFrame to TimeSeries and split target and predictors
predictors_names = ["x" + str(x + 1) for x in range(df.shape[1]-1)] # allows that the number of predictors can be generic; ensure in data preprocessing that target is named 'y' and predictors 'x1', 'x2' etc.
df[predictors_names] = df[predictors_names].shift(1)
df.dropna(inplace=True)


# Split the data into training and validation sets
train_split = 0.3
train_size = int(df.shape[0] * train_split)
train = df[:train_size]

fc_horizon = 1
current_predictors = df[predictors_names][:(train_size+fc_horizon)]

In [159]:
print(train)

                 y          x1         x2          x3
Date                                                 
2004-02  52.871538  427.595799  55.337904  900.325291
2004-03  53.769316  434.062163  54.959155  900.775888
2004-04  57.672973  453.264284  56.470633  899.510058
2004-05  57.182051  459.367523  56.704233  903.524834
2004-06  58.006999  462.354356  61.557907  905.071762
...            ...         ...        ...         ...
2009-08  60.224217  403.591808  44.461474  918.266042
2009-09  60.550782  412.073922  43.591783  923.096722
2009-10  59.701182  418.360799  43.497459  917.592521
2009-11  62.274049  402.146132  41.007900  918.810155
2009-12  59.512417  405.157508  42.137943  918.435673

[71 rows x 4 columns]


In [70]:
use_predictors_bool = True
lag_predictors = False

In [160]:
# Setting up Auto-SARIMAX Model without covariates
model = AutoARIMA(
    seasonal=True,
    sp=12,
    trace=True,
    with_intercept="auto",
    max_p=12,
    max_q=12,
    suppress_warnings=True
)


In [171]:
# Train Model on Training Data
model.fit(train['y'], train[predictors_names])

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(1,0,1)[12] intercept   : AIC=311.377, Time=1.05 sec
 ARIMA(0,0,0)(0,0,0)[12] intercept   : AIC=389.612, Time=0.09 sec
 ARIMA(1,0,0)(1,0,0)[12] intercept   : AIC=331.440, Time=0.55 sec
 ARIMA(0,0,1)(0,0,1)[12] intercept   : AIC=369.772, Time=0.58 sec
 ARIMA(0,0,0)(0,0,0)[12]             : AIC=387.622, Time=0.19 sec
 ARIMA(2,0,2)(0,0,1)[12] intercept   : AIC=309.230, Time=1.08 sec
 ARIMA(2,0,2)(0,0,0)[12] intercept   : AIC=308.733, Time=0.59 sec
 ARIMA(2,0,2)(1,0,0)[12] intercept   : AIC=309.114, Time=0.96 sec
 ARIMA(1,0,2)(0,0,0)[12] intercept   : AIC=307.367, Time=0.37 sec
 ARIMA(1,0,2)(1,0,0)[12] intercept   : AIC=308.095, Time=0.88 sec
 ARIMA(1,0,2)(0,0,1)[12] intercept   : AIC=308.177, Time=1.03 sec
 ARIMA(1,0,2)(1,0,1)[12] intercept   : AIC=310.256, Time=1.42 sec
 ARIMA(0,0,2)(0,0,0)[12] intercept   : AIC=336.845, Time=0.43 sec
 ARIMA(1,0,1)(0,0,0)[12] intercept   : AIC=328.078, Time=0.42 sec
 ARIMA(1,0,3)(0,0,0)[12] intercep

AutoARIMA(max_p=12, max_q=12, sp=12, suppress_warnings=True, trace=True,
          with_intercept='auto')

In [162]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                   71
Model:             SARIMAX(1, 0, 2)x(0, 0, [1], 12)   Log Likelihood                -150.281
Date:                              Wed, 21 Feb 2024   AIC                            316.562
Time:                                      22:46:39   BIC                            334.664
Sample:                                  02-29-2004   HQIC                           323.760
                                       - 12-31-2009                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0135      0.026     -0.520      0.603      -0.064       0.037
x2             0.2371      0.165      1.441      0.150      -0.085       0.560
x3             0.0507      0.016      3.192      0.001       0.020       0.082
ar.L1          0.9346      0.050     18.705      0.000       0.837       1.033
ma.L1         -0.3430      0.121     -2.832      0.005      -0.580      -0.106
ma.L2          0.6005      0.112      5.349      0.000       0.381       0.821
ma.S.L12       0.1613      0.125      1.294      0.196      -0.083       0.406
sigma2         3.8215      0.778      4.909      0.000       2.296       5.347
===================================================================================
Ljung-Box (L1) (Q):                   0.11   Jarque-Bera (JB):                 0.31
Prob(Q):                              0.74   Prob(JB):                         0.86
Heteroskedasticity (H):               0.88   Skew:                             0.08
Prob(H) (two-sided):                  0.77   Kurtosis:                         2.72
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [163]:
train[:train_size]

,y,x1,x2,x3
Date,,,,
2004-02,52.871538,427.595799,55.337904,900.325291
2004-03,53.769316,434.062163,54.959155,900.775888
2004-04,57.672973,453.264284,56.470633,899.510058
2004-05,57.182051,459.367523,56.704233,903.524834
2004-06,58.006999,462.354356,61.557907,905.071762
...,...,...,...,...
2009-08,60.224217,403.591808,44.461474,918.266042
2009-09,60.550782,412.073922,43.591783,923.096722
2009-10,59.701182,418.360799,43.497459,917.592521


In [181]:
current_predictors

,x1,x2,x3
Date,,,
2004-02,427.595799,55.337904,900.325291
2004-03,434.062163,54.959155,900.775888
2004-04,453.264284,56.470633,899.510058
2004-05,459.367523,56.704233,903.524834
2004-06,462.354356,61.557907,905.071762
...,...,...,...
2009-09,412.073922,43.591783,923.096722
2009-10,418.360799,43.497459,917.592521
2009-11,402.146132,41.007900,918.810155


In [182]:
print(train)

                 y          x1         x2          x3
Date                                                 
2004-02  52.871538  427.595799  55.337904  900.325291
2004-03  53.769316  434.062163  54.959155  900.775888
2004-04  57.672973  453.264284  56.470633  899.510058
2004-05  57.182051  459.367523  56.704233  903.524834
2004-06  58.006999  462.354356  61.557907  905.071762
...            ...         ...        ...         ...
2009-08  60.224217  403.591808  44.461474  918.266042
2009-09  60.550782  412.073922  43.591783  923.096722
2009-10  59.701182  418.360799  43.497459  917.592521
2009-11  62.274049  402.146132  41.007900  918.810155
2009-12  59.512417  405.157508  42.137943  918.435673

[71 rows x 4 columns]


In [185]:
prediction = model.predict(fh=1, X=current_predictors)

ValueError: X array dims (n_rows) != n_periods. Received n_rows=0 and n_periods=1

In [174]:
print(prediction)

2010-03    62.195747
Freq: M, Name: y, dtype: float64
